In [390]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [391]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [392]:
#DISTRICT SUMMARY [CFH]
# Calculate the total number of schools
num_school = len(school_data_complete['school_name'].unique())
# num_school

# Calculate the total number of students
num_student = len(school_data_complete['student_name'].unique())
# num_student


# Calculate the total budget
total_budget = school_data_complete['budget'].sum()
#total_budget

# Calculate the average math score
avg_math = school_data_complete['math_score'].mean()
avg_math = round(avg_math, 2)

# Calculate the average reading score
avg_read = school_data_complete['reading_score'].mean()
avg_read = round(avg_read, 2)

# Calculate the percentage of students with a passing math score (70 or greater)
pass_math = len(school_data_complete.loc[school_data_complete['math_score'] >= 70])
all_math = len(school_data_complete.loc[school_data_complete['math_score']])
percent_math = round((pass_math/all_math)*100, 2)
# percent_math

# Calculate the percentage of students with a passing reading score (70 or greater)
pass_read = len(school_data_complete.loc[school_data_complete['reading_score'] >= 70])
all_read= len(school_data_complete.loc[school_data_complete['reading_score']])
percent_read = round((pass_read/all_read)*100, 2)
#percent_read

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_pass = round((percent_math+percent_read)/2, 2)



# Create a dataframe to hold the above results
frame_df = pd.DataFrame({
    "Total Schools": [num_school],
    "Total Students": [num_student],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math],
    "Average Reading Score": [avg_read],
    "% Passing Math": [percent_math],
    "% Passing Reading": [percent_read],
    "% Overall Passing Rate": [overall_pass]

})


# Optional: give the displayed data cleaner formatting
frame_df['Total Students'] = frame_df.apply(lambda x: "{:,}".format(x['Total Students']), axis=1)
frame_df['Total Budget'] = frame_df.apply(lambda x: "${:,.2f}".format(x['Total Budget']), axis=1)

frame_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"32,715.0","$82,932,329,558.00",78.99,81.88,74.98,85.81,80.4


In [464]:
# #SCHOOL SUMMARY [CFH]
# # Create an overview table that summarizes key metrics about each school, including:
# School Type
# Total School Budget
data_complete = school_data_complete.rename(columns={'student_name': 'Student Name',
                                                     'gender' : 'Gender', 
                                                      'grade' : 'Grade',
                                                       'school_name' : 'School Name',
                                                        'reading_score': 'Reading Score',
                                                        'math_score': 'Math Score'
                                                    })
school = school_data.drop(['School ID'], axis=1)
school.rename(columns={'school_name' : 'School Name',
                      'type' : 'School Type', 
                      'size' : 'Total Students',
                       'budget' : 'Total Budget'
                      }, inplace=True)
student_data2 = student_data.drop(['Student ID'], axis=1)
student_data2.rename(columns={'student_name' : 'Student Name',
                      'gender' : 'Gender', 
                      'grade' : 'Grade',
                       'school_name' : 'School Name',
                        'reading_score': 'Reading Score',
                        'math_score': 'Math Score'
                      }, inplace=True)

# Per Student Budget
budget = school_data['budget']
# print(budget)
students = school_data['size']
# print(students)
per_student= budget/students
# print(per_student)
per_student_df = pd.DataFrame(per_student)
# per_student_df
school.insert(4, "Budget Per Student", per_student_df, True)
by_school = school.set_index("School Name")
# by_school

# Average Math Score
students_by_school = student_data2.groupby(['School Name'])
sum_students = students_by_school.mean()
sum_students = round(sum_students, 2)
sum_students.rename(columns={'Reading Score': 'Average Reading Score',
                            'Math Score' : "Average Math Score"})


# Average Reading Score

# % Passing Math
stu = data_complete.groupby('School Name')
stu = stu["Student ID"].count()
stu_df = pd.DataFrame(stu)

passing_math = data_complete.loc[data_complete['Math Score'] >= 70, ["School Name", 'Math Score']]
passing_math = passing_math.groupby(['School Name'])
spassing_math = passing_math.count()
spassing_math.insert(1, "STU", stu_df, True)
spassing_math
pm_total = spassing_math['Math Score']
am_total =spassing_math['STU']
avg_pm = round((pm_total/am_total)*100, 2)
passing_math_df = pd.DataFrame(avg_pm)
passing_math_df

                                  
passing_reading = data_complete.loc[data_complete['Reading Score'] >= 70, ["School Name", 'Reading Score']]
passing_reading = passing_reading.groupby(['School Name'])
spassing_reading = passing_reading.count()
spassing_reading.insert(1, "STU", stu_df, True) 
spassing_reading
pr_total = spassing_reading["Reading Score"]
ar_total = spassing_reading["STU"]
avg_pr = round((pr_total/ar_total)*100, 2)
passing_reading_df = pd.DataFrame(avg_pr)


# % Passing Reading
# Overall Passing Rate (Average of the above two)
overall_passing = round((avg_pr+avg_pm)/2,2)
overall_passing_avg_df = pd.DataFrame(overall_passing)
overall_passing_avg_df
# Create a dataframe to hold the above results

by_school1 = pd.merge(by_school, sum_students, how="outer", on=["School Name"])
by_school1.insert(5, "% Passing Reading", passing_reading_df, True)
by_school1.insert(7, "% Passing Math", passing_math_df, True)
by_school2 = pd.merge(by_school1, overall_passing_avg_df, how='left', on=["School Name"])

Final = by_school2.rename(columns={'Reading Score' : 'Average Reading Score',
                          'Math Score' : 'Average Math Score',
                          0 : 'Overall Passing Average'})
Final

,School Type,Total Students,Total Budget,Budget Per Student,Average Reading Score,% Passing Reading,Average Math Score,% Passing Math,Overall Passing Average
School Name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,81.18,81.32,76.63,65.68,73.50
Figueroa High School,District,2949,1884411,639.0,81.16,80.74,76.71,65.99,73.36
Shelton High School,Charter,1761,1056600,600.0,83.73,95.85,83.36,93.87,94.86
Hernandez High School,District,4635,3022020,652.0,80.93,80.86,77.29,66.75,73.81
Griffin High School,Charter,1468,917500,625.0,83.82,97.14,83.35,93.39,95.26
Wilson High School,Charter,2283,1319574,578.0,83.99,96.54,83.27,93.87,95.21
Cabrera High School,Charter,1858,1081356,582.0,83.98,97.04,83.06,94.13,95.58
Bailey High School,District,4976,3124928,628.0,81.03,81.93,77.05,66.68,74.31
Holden High School,Charter,427,248087,581.0,83.81,96.25,83.80,92.51,94.38


In [465]:
Final_sort = Final.sort_values(
    ["Overall Passing Average"], ascending=False)
Final_sort.head(5)


,School Type,Total Students,Total Budget,Budget Per Student,Average Reading Score,% Passing Reading,Average Math Score,% Passing Math,Overall Passing Average
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.98,97.04,83.06,94.13,95.58
Thomas High School,Charter,1635,1043130,638.0,83.85,97.31,83.42,93.27,95.29
Pena High School,Charter,962,585858,609.0,84.04,95.95,83.84,94.59,95.27
Griffin High School,Charter,1468,917500,625.0,83.82,97.14,83.35,93.39,95.26
Wilson High School,Charter,2283,1319574,578.0,83.99,96.54,83.27,93.87,95.21


In [466]:
Final_sort = Final.sort_values(
    ["Overall Passing Average"])
Final_sort.head(5)

,School Type,Total Students,Total Budget,Budget Per Student,Average Reading Score,% Passing Reading,Average Math Score,% Passing Math,Overall Passing Average
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,80.74,80.22,76.84,66.37,73.30
Figueroa High School,District,2949,1884411,639.0,81.16,80.74,76.71,65.99,73.36
Huang High School,District,2917,1910635,655.0,81.18,81.32,76.63,65.68,73.50
Johnson High School,District,4761,3094650,650.0,80.97,81.22,77.07,66.06,73.64
Hernandez High School,District,4635,3022020,652.0,80.93,80.86,77.29,66.75,73.81


In [467]:
# #Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
d = student_data.drop(['Student ID', 'student_name', 'gender', 'reading_score'], axis = 1)
d.head()
math_by_grade = d.groupby(['school_name', 'grade'])
round(math_by_grade.mean(),2)

math_score
school_name           grade            
Bailey High School    10th        77.00
                      11th        77.52
                      12th        76.49
                      9th         77.08
Cabrera High School   10th        83.15
                      11th        82.77
                      12th        83.28
                      9th         83.09
Figueroa High School  10th        76.54
                      11th        76.88
                      12th        77.15
                      9th         76.40
Ford High School      10th        77.67
                      11th        76.92
                      12th        76.18
                      9th         77.36
Griffin High School   10th        84.23
                      11th        83.84
                      12th        83.36
                      9th         82.04
Hernandez High School 10th        77.34
                      11th        77.14
                      12th        77.19
                      9th         77.44
Holden High School    10th        83.43
                      11th        85.00
                      12th        82.86
                      9th         83.79
Huang High School     10th        75.91
                      11th        76.45
                      12th        77.23
                      9th         77.03
Johnson High School   10th        76.69
                      11th        77.49
                      12th        76.86
                      9th         77.19
Pena High School      10th        83.37
                      11th        84.33
                      12th        84.12
                      9th         83.63
Rodriguez High School 10th        76.61
                      11th        76.40
                      12th        77.69
                      9th         76.86
Shelton High School   10th        82.92
                      11th        83.38
                      12th        83.78
                      9th         83.42
Thomas High School    10th        83.09
                      11th        83.50
                      12th        83.50
                      9th         83.59
Wilson High School    10th        83.72
                      11th        83.20
                      12th        83.04
                      9th         83.09
Wright High School    10th        84.01
                      11th        83.84
                      12th        83.64
                      9th         83.26

In [468]:
#* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
dd = student_data.drop(['Student ID', 'student_name', 'gender', 'math_score'], axis = 1)
dd.head()
reading_by_grade = dd.groupby(['school_name', 'grade'])
round(reading_by_grade.mean(),2)

reading_score
school_name           grade               
Bailey High School    10th           80.91
                      11th           80.95
                      12th           80.91
                      9th            81.30
Cabrera High School   10th           84.25
                      11th           83.79
                      12th           84.29
                      9th            83.68
Figueroa High School  10th           81.41
                      11th           80.64
                      12th           81.38
                      9th            81.20
Ford High School      10th           81.26
                      11th           80.40
                      12th           80.66
                      9th            80.63
Griffin High School   10th           83.71
                      11th           84.29
                      12th           84.01
                      9th            83.37
Hernandez High School 10th           80.66
                      11th           81.40
                      12th           80.86
                      9th            80.87
Holden High School    10th           83.32
                      11th           83.82
                      12th           84.70
                      9th            83.68
Huang High School     10th           81.51
                      11th           81.42
                      12th           80.31
                      9th            81.29
Johnson High School   10th           80.77
                      11th           80.62
                      12th           81.23
                      9th            81.26
Pena High School      10th           83.61
                      11th           84.34
                      12th           84.59
                      9th            83.81
Rodriguez High School 10th           80.63
                      11th           80.86
                      12th           80.38
                      9th            80.99
Shelton High School   10th           83.44
                      11th           84.37
                      12th           82.78
                      9th            84.12
Thomas High School    10th           84.25
                      11th           83.59
                      12th           83.83
                      9th            83.73
Wilson High School    10th           84.02
                      11th           83.76
                      12th           84.32
                      9th            83.94
Wright High School    10th           83.81
                      11th           84.16
                      12th           84.07
                      9th            83.83

In [469]:
# * Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
#   * Average Math Score
#   * Average Reading Score
#   * % Passing Math
#   * % Passing Reading
#   * Overall Passing Rate (Average of the above two)
bin_labels = ['0%-25%', '26%-50%', '51%-75%', '76%-100%']
Final1 = Final
Final1['Spending Ranges Per Student'] = pd.qcut(Final1['Budget Per Student'],
                              q=[0, .25, .5, .75, 1],
                              labels=bin_labels)
Final1 = Final1.groupby(['Spending Ranges Per Student'])
Final1.mean()
# Final['Budget Per Student'].describe()

,Total Students,Total Budget,Budget Per Student,Average Reading Score,% Passing Reading,Average Math Score,% Passing Math,Overall Passing Average
Spending Ranges Per Student,,,,,,,,
0%-25%,1592.00,924604.25,581.00,83.935000,96.6100,83.4525,93.4600,95.035
26%-50%,2291.75,1421221.50,615.50,83.155000,92.7175,81.9000,87.1325,89.925
51%-75%,2861.00,1824968.00,638.00,81.916667,86.0900,78.9900,75.2100,80.650
76%-100%,3763.00,2447805.25,650.25,80.957500,80.6750,77.0225,66.7000,73.690


In [470]:
# * Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).

bin_labels_size = ['Small', 'Medium', 'Large']
Final2 = Final
Final2['School Size'] = pd.qcut(Final2['Total Students'],
                              q=3,
                              labels=bin_labels_size)
Final2 = Final2.groupby(['School Size'])
Final2.mean()


,Total Students,Total Budget,Budget Per Student,Average Reading Score,% Passing Reading,Average Math Score,% Passing Math,Overall Passing Average
School Size,,,,,,,,
Small,1250.6,770235.0,610.6,83.850,96.500,83.554,93.526,95.012
Medium,2319.4,1424976.2,608.4,82.772,90.162,80.748,83.064,86.614
Large,4264.0,2734674.4,641.2,80.966,80.994,76.992,66.370,73.684


In [471]:
# Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

Final3 = Final.groupby(['School Type'])
Final3.mean()



,Total Students,Total Budget,Budget Per Student,Average Reading Score,% Passing Reading,Average Math Score,% Passing Math,Overall Passing Average
School Type,,,,,,,,
Charter,1524.250000,9.126881e+05,599.500000,83.897500,96.586250,83.472500,93.620000,95.102500
District,3853.714286,2.478275e+06,643.571429,80.965714,80.798571,76.955714,66.548571,73.675714
